In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
tokenizer = AutoTokenizer.from_pretrained("abhibisht89/spanbert-large-cased-finetuned-ade_corpus_v2")
model = AutoModelForTokenClassification.from_pretrained("abhibisht89/spanbert-large-cased-finetuned-ade_corpus_v2").to('cpu')
adr_ner_model = pipeline(task="ner", model=model, tokenizer=tokenizer, grouped_entities=True)

tokenizer_config.json:   0%|          | 0.00/356 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/887 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


In [3]:
df = pd.read_csv('/kaggle/input/model1ds/model1_adhd_output.csv')

In [4]:
def get_ade_from_text(sentence):
    tokens = tokenizer.tokenize(sentence)    
    chunks = [tokens[i:i+500] for i in range(0, len(tokens), 512)]    
    adverse_effects = []    
    for chunk in chunks:        
        chunk_text = tokenizer.convert_tokens_to_string(chunk)        
        # Apply ADE model to each chunk        
        tokens = adr_ner_model(chunk_text)        
        # Extract adverse effects from each chunk       
        adverse_effects.extend([token["word"] for token in tokens if token["entity_group"] == "ADR"])    
        # Return all extracted ADEs as a comma-separated string, or empty if none    
    return ", ".join(adverse_effects) if adverse_effects else ""

In [5]:
df['extracted_ade'] = df['text'].apply(get_ade_from_text)
print(df.head())

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


                                                text  predictions  \
0  I know this will be hard for you to read as I ...            1   
1  I tried adderall for the first time last year,...            1   
2  I don't have ADD or ADHD (to my knowledge). I ...            1   
3  Basically, I saw my doctor on Friday, and inst...            1   
4  I'm planning on going to the doctor tomorrow t...            1   

  extracted_ade  
0                
1      dry rash  
2        sleepy  
3                
4                


In [6]:
df.to_csv('model2_adhd_output.csv', index=False)